# Word Count Analysis

In [1]:
#import libraries
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
#import dataframes
df_cty = pd.read_csv('df_cty_final.csv',converters={'words': eval})
df_rb = pd.read_csv('df_rb_final.csv',converters={'words': eval})
df_rock = pd.read_csv('df_rock_final.csv',converters={'words': eval})

### Functions

In [3]:
def word_count(word_list):
    '''
    function
    parameters:
        word_list-->list of word lists for each song in dataframe
    '''
    #initiate Counter objects
    word_counts = Counter() #number of times a word appears in all songs
    appears_in = Counter() #number of songs in which a word appears
    
    #iterate through lists of words in each song to update variables
    for l in word_list:
        word_counts.update(l)
        appears_in.update(set(l))
    
    #create new dataframe with word, count, and appears_in columns
    df = pd.DataFrame(data={'word':word_counts.keys(),
                            'count':word_counts.values(),
                            'appears_in':appears_in.values()})
    
    #add percent total column (based on total word count)
    df['count_pct'] = df['count'].apply(lambda x: x/df['count'].sum())
    
    #add appears in percent column (based on total number of songs)
    df['appears_in_pct'] = df['appears_in'].apply(lambda x: x/len(word_list))
    
    return df.sort_values(by='count',ascending=False).set_index('word')

In [4]:
def remove_num(text):
    '''
    function
    parameter:
        text-->
    '''
    for word in text:
        try:
            int(word)
            text.remove(word)
        except:
            pass
    return text

In [5]:
#remove numbers from dataframs
df_cty['words'] = df_cty['words'].apply(remove_num)
df_rb['words'] = df_rb['words'].apply(remove_num)
df_rock['words'] = df_rock['words'].apply(remove_num)

In [6]:
#create dataframes with word counts
df_cty_count = word_count(df_cty['words'])
df_rb_count = word_count(df_rb['words'])
df_rock_count = word_count(df_rock['words'])

In [7]:
df_cty_count

,count,appears_in,count_pct,appears_in_pct
word,,,,
like,3485,26,0.011275,0.025616
know,2785,118,0.009011,0.116256
yeah,2538,30,0.008212,0.029557
love,2331,142,0.007542,0.139901
one,2194,13,0.007099,0.012808
...,...,...,...,...
enterprise,1,2,0.000003,0.001970
peopleit,1,1,0.000003,0.000985
satyriasis,1,1,0.000003,0.000985
